# Home Work 3. Логистическая регрессия. Log Loss

Для выполнения домашнего задания будем использовать файл Lesson_3_extended.ipynb из материалов к уроку:

In [1]:
import numpy as np
import matplotlib.pyplot as plt


In [2]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype=np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

In [3]:
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

In [4]:
X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])

In [5]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [6]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [7]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
#         if i % (iterations / 10) == 0:
#             print(i, W, err)
    return W

### Задача 1*. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

Для решения задачи, применим функцию **np.clip**.

In [8]:
def calc_logloss(y, y_pred):
    err = np.mean(- y * np.log(np.clip(y_pred, a_min=0.00001, a_max=1)) - \
                  (1.0 - y) * np.log(np.clip(1.0 - y_pred, a_min=0.00001, a_max=1)))
    return err

In [9]:
print(calc_logloss(np.array([0, 1]), np.array([0, 1])))

0.0


### Задача 2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.


In [10]:
iterations_list = [1000, 5000, 10000, 15000, 20000, 100000]
eta_list = [1e-3, 1e-4, 1e-5, 0.002, 0.003, 0.004]

best_error = np.inf
best_params = {}

for i in iterations_list:
    for j in eta_list:
        np.random.seed(42)
        W = np.random.randn(X_st.shape[1])
        n = X_st.shape[0]
    
        for i in range(i+1):
            z = np.dot(X_st, W)
            y_pred = sigmoid(z)
            err = calc_logloss(y, y_pred)
        
            dQ = 1/n * X_st.T @ (y_pred - y)
            W -= j * dQ
        if err < best_error:
            best_error = err
            best_params = {
                    'eta': j,
                    'n_iter': i
            }
                    
print(f'\nЛучшая ошибка {best_error} с параметрами {best_params}')


Лучшая ошибка 0.2825319832255081 с параметрами {'eta': 0.004, 'n_iter': 100000}


### Задача 3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [11]:
W = eval_model(X_st, y, iterations = 100000, eta=0.004)
W

array([-6.58491989, -1.08568691, -1.09077503,  5.8979015 ])

In [12]:
def calc_pred_proba(x, w):
    z = np.dot(x, w) # значения функции линейной регрессии
    y_pred_proba = sigmoid(z) # находим предсказанные вероятности
    return list(y_pred_proba)

In [13]:
print(calc_pred_proba(X_st, W))

[0.3308831702513757, 0.23974225517352413, 0.9719590174515729, 0.005107671641483346, 0.7079214562546317, 0.42718244211706785, 0.9890061348157567, 0.11366687123949587, 0.35624067525002673, 0.9518126561185714]


### Задача 4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [14]:
def calc_pred(x, w):
    z = np.dot(x, w) # значения функции линейной регрессии
    y_pred_proba = sigmoid(z) # находим предсказанные вероятности
    y_pred = [1 if y > 0.5 else 0 for y in y_pred_proba] # переводим в 0 и 1
    return np.array(y_pred)

In [15]:
print(calc_pred(X_st, W))
print(y.astype(int))

[0 0 1 0 1 0 1 0 0 1]
[0 0 1 0 1 0 1 0 1 1]


### Задача 5*. Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.

In [16]:
y_pred = calc_pred(X_st, W)

**Accuracy**  
(доля правильных ответов):

$$accuracy(a,x) = \frac{1}{l} \sum^{l}_{i=1}[a(x_{i})=y_{i}].$$

In [17]:
def score_accuracy(y_pred, y):
    acc = np.sum((y_pred == y)+0) / len(y)
    return str(round(acc*100, 1)) + '%'

In [18]:
score_accuracy(y_pred, y)

'90.0%'

**Матрица ошибок**  

|  <empty>   | $$y = +1$$ | $$y = -1$$ |
--- | --- | ---
| __$$a(x) = +1$$__  |  True Positive TP    |  False Positive  FP   |
| __$$a(x) = -1$$__ |   False Negative FN    |   True Negative TN   |

In [19]:
def score_confusion_matrix(y_pred, y):
    err_matrix = np.zeros((2, 2))
    for i in range(len(y)):
        if y_pred[i] == 1 and y[i] == 1:
            err_matrix[0][0] += 1
        elif y_pred[i] == 1 and y[i] != 1:
            err_matrix[0][1] += 1
        elif y_pred[i] == 0 and y[i] == 0:
            err_matrix[1][1] += 1
        else:
            err_matrix[1][0] += 1
    return err_matrix

In [20]:
print(score_confusion_matrix(y_pred, y))

[[4. 0.]
 [1. 5.]]


**Точность**  
Точность (precision) представляет из себя долю истинных срабатываний от общего количества срабатываний. Она показывает, насколько можно доверять алгоритму классификации в случае срабатывания

$$precision(a, X) = \frac{TP}{TP+FP}.$$

In [21]:
def precision_score(y_pred, y):    
    TP = sum([1 if i[0] == i[1] == 1 else 0 for i in zip(y, y_pred)])
    FP = sum([1 if i[0] == 1 and i[1] == 0 else 0 for i in zip(y, y_pred)])
    precision = TP / (TP + FP)
    return precision

In [22]:
precision_score(y_pred, y)

0.8

**Полнота**  
Полнота (recall) считается как доля объектов, истинно относящихся к классу "+1", которые алгоритм отнес к этому классу

$$recall(a, X) = \frac{TP}{TP+FN},$$

здесь $TP+FN$ как раз будут вместе составлять весь список объектов класса "+1".

In [23]:
def recall_score(y_pred, y):
    TP = sum([1 if i[0] == i[1] == 1 else 0 for i in zip(y, y_pred)])
    TP_FN = sum(y == 1)
    recall = TP / TP_FN
    return recall

In [24]:
recall_score(y_pred, y)

0.8

**F1 score**  
F-мера представляет собой среднее гармоническое между точностью и полнотой

$$F = \frac{2 \cdot precision \cdot recall }{ presision + recall}.$$

In [25]:
def F1_score(y_pred, y):
    TP = sum([1 if i[0] == i[1] == 1 else 0 for i in zip(y, y_pred)])
    FP = sum([1 if i[0] == 1 and i[1] == 0 else 0 for i in zip(y, y_pred)])
    TP_FN = sum(y == 1)
    precision = TP / (TP + FP)
    recall = TP / TP_FN
    F1_score = (2 * precision * recall) / (precision + recall)
    return F1_score

In [26]:
F1_score(y_pred, y)

0.8000000000000002

### Задача 6. Могла ли модель переобучиться? Почему?

**Ответ**  
Модель могла переобучиться, так как в датасете слишком мало наблюдений.